File Name: orginal_labels_classification

Description: This notebook experiments with difference ML models to find which is best at predicting recipes based on the orignal data. The best model will then be used to assess the accuracy of generated samples from the autoencoder.  

In [2]:
import tensorflow as tf
import keras
import numpy as np
from sklearn.model_selection import train_test_split 
import string
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

2025-07-18 21:56:53.210840: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-18 21:56:53.276012: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-18 21:56:53.276065: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-18 21:56:53.277547: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-18 21:56:53.285114: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-18 21:56:53.286090: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
df = pd.read_csv("../data/reduced_gin_data.csv")
flavour = df[df.columns[6:23]]
botanicals = df[df.columns[25:278]]
compounds = df[df.columns[278:]]
for column in compounds.columns:
    compounds[column] = (compounds[column] - compounds[column].min()) / (compounds[column].max() - compounds[column].min())
df.__len__()

/tmp/ipykernel_235684/838367829.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compounds[column] = (compounds[column] - compounds[column].min()) / (compounds[column].max() - compounds[column].min())


329

In [85]:
flavour

,Coffee (F),Nutty (F),Mixed Fruit (F),Cacao (F),Herbal (F),Herbaceous (F),Stone Fruit (F),Rhubarb (F),Sweet (F),Tropical Fruit (F),Black Fruit (F),Aged (F),Green Fruit (F),Floral (F),Spice (F),Citrus Fruit (F),Red Fruit (F)
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
325,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
326,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
327,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0


## botanical - flavour classifier 

# NN

In [4]:
#input layer
inputs = keras.Input(flavour.columns.__len__()) 
#hidden layers
h1 = keras.layers.Dense((flavour.columns.__len__()*2), activation='relu')
h2 = keras.layers.Dense(flavour.columns.__len__(), activation='relu')

#construct input and hidden layers
layer = h1(inputs)
layer = h2(layer)



#parse the botanical names to get rid of spaces and punct ready to set as output layer names
translator = str.maketrans('', '', string.punctuation)
layer_names = ["".join(botanical.split()).translate(translator) for botanical in botanicals.columns]

#contruct output layers
output_neurons = np.array(range(len(layer_names)), dtype=keras.layers.Dense)

for index, botanical in enumerate(layer_names):
    output_neurons[index] = keras.layers.Dense(1, activation='sigmoid', name=botanical)(layer)

    
#create model
model = keras.models.Model(inputs = inputs, outputs = output_neurons.tolist(), name = "Botanical_pred_by_flavour")
model.build(flavour.__len__())

#for multi-layer ouput architecture
model.compile(
    optimizer='adam',
    loss = {botanical : "binary_crossentropy" for botanical in layer_names},
    metrics = {botanical : ["F1", "Specificity"] for botanical in layer_names},
    run_eagerly=True
)
model.summary()

Model: "Botanical_pred_by_flavour"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 17)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 34)                   612       ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 17)                   595       ['dense[0][0]']               
                                                                                                  
 Rosewater (Dense)           (None, 1)                    18        ['dense_1[0][0]']             
                                                                          

In [5]:
#input layer
inputs2 = keras.Input(botanicals.columns.__len__()) 
#hidden layers
h1_2 = keras.layers.Dense(botanicals.columns.__len__()*2, activation='relu')
h2_2 = keras.layers.Dense(botanicals.columns.__len__(), activation='relu')
h3_2 = keras.layers.Dense(round(botanicals.columns.__len__()/2), activation='relu')
output = keras.layers.Dense(flavour.columns.__len__(), activation='sigmoid')

#construct input and hidden layers
x = h1_2(inputs2)
x = h2_2(x)
#x = h3_2(x)
x = output(x)

model2 = keras.models.Model(inputs = inputs2, outputs = x, name = "Botanical_pred_by_flavour_single_output_layer")

model2.build(botanicals.columns.__len__())

model2.compile(
    optimizer=keras.optimizers.Adam(),
    loss = keras.losses.BinaryCrossentropy(),
    metrics = keras.metrics.BinaryAccuracy(),
    run_eagerly=True
)

model2.summary()


Model: "Botanical_pred_by_flavour_single_output_layer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 253)]             0         
                                                                 
 dense_2 (Dense)             (None, 506)               128524    
                                                                 
 dense_3 (Dense)             (None, 253)               128271    
                                                                 
 dense_5 (Dense)             (None, 17)                4318      
                                                                 
Total params: 261113 (1019.97 KB)
Trainable params: 261113 (1019.97 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# x = np.array(range(len(flavour)), dtype=list)
# y = np.array(range(len(botanicals)), dtype=list)

x = []#np.array([np.zeros(flavour.columns.__len__()) for x in range(flavour.__len__())])
# y = np.zeros(botanicals.columns.__len__()).tolist()
# y = [[] for _ in y]
y = []#np.array([np.zeros(botanicals.columns.__len__()) for x in range(botanicals.__len__())])

#create x and y arrays
for index, row in flavour.iterrows():
    y.append(row.values.tolist())
    
    
for index_1, row in botanicals.iterrows():
    x.append(row.values.tolist())
    #y.append(((x) for x in row.values.tolist()))
    # for index_2, value in enumerate(row.values.tolist()):
    #     y[index_2].append(value)

type(x)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.5)


In [7]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=0.01)
checkpoint = tf.keras.callbacks.ModelCheckpoint('checkpoint.keras', monitor='val_loss', save_best_only=True)
model2.fit(X_train, y_train, validation_data=(X_val, y_val), verbose=True, epochs=150, callbacks=[callback, checkpoint])

Epoch 1/150
4/4 [==============================] - 1s 141ms/step - loss: 0.6727 - binary_accuracy: 0.6547 - val_loss: 0.6137 - val_binary_accuracy: 0.9090
Epoch 2/150
4/4 [==============================] - 0s 104ms/step - loss: 0.5767 - binary_accuracy: 0.9197 - val_loss: 0.5106 - val_binary_accuracy: 0.9238
Epoch 3/150
4/4 [==============================] - 0s 100ms/step - loss: 0.4665 - binary_accuracy: 0.9243 - val_loss: 0.3986 - val_binary_accuracy: 0.9238
Epoch 4/150
4/4 [==============================] - 1s 155ms/step - loss: 0.3620 - binary_accuracy: 0.9243 - val_loss: 0.3184 - val_binary_accuracy: 0.9238
Epoch 5/150
4/4 [==============================] - 0s 91ms/step - loss: 0.2980 - binary_accuracy: 0.9243 - val_loss: 0.2873 - val_binary_accuracy: 0.9238
Epoch 6/150
4/4 [==============================] - 0s 90ms/step - loss: 0.2732 - binary_accuracy: 0.9243 - val_loss: 0.2798 - val_binary_accuracy: 0.9238
Epoch 7/150
4/4 [==============================] - 0s 90ms/step - loss: 

In [8]:

pred_tensor = model2.predict(X_test).round()
#pred_tensor = []
# for p in pred:
#     tensor = np.zeros(len(pred[0]))
#     tensor[p.argmax()] = 1
#     pred_tensor.append(tensor)
print(f'accuracy score {accuracy_score(y_test, pred_tensor)}')
#print(f'balanced accuracy score {balanced_accuracy_score(y_test, pred)}')
print(f'precision {precision_score(y_test, pred_tensor,average=None)}')
print(f'recall {recall_score(y_test, pred_tensor, average=None)}')



4/4 [==============================] - 0s 5ms/step
accuracy score 0.3434343434343434
precision [0.         0.         0.         0.         0.         1.
 1.         1.         0.         0.         1.         0.
 1.         0.93333333 1.         0.88888889 0.90909091]
recall [0.         0.         0.         0.         0.         0.25
 0.28571429 0.625      0.         0.         0.5        0.
 0.375      0.77777778 0.16666667 0.44444444 0.625     ]


/home/michael/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/michael/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
counter = np.zeros(17)
for i in y_test: 
    y = np.where(np.array(i) == 1)
    for w in y: 
        counter[w] += 1
        
print(counter)

[ 0.  0.  1.  0.  5.  4.  7.  8.  8.  7.  8.  1.  8. 18. 18. 18. 16.]


# desicion tree

In [75]:
params = {
    #compare both split evaluation functions
    'criterion' : ['gini', 'entropy'], 
    
    #after some research it's was found a tree in this depth range is approate for a database of this size 
    'max_depth' : [120, 150, 180, 200, 250],  
    
    #this describes what % of each class should be at each leaf node, a higher value here can be used to limit overfitting
    'min_weight_fraction_leaf' : [0.0, 0.1, 0.2, 0.3], 
    
    #this decides if a split is allowed based on how many samples are at that node
    'min_samples_split' : [2, 3, 4, 5, 6, 7, 8, 9, 10]
}


gs_decision_tree = GridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid=params,
    scoring='accuracy',
    cv=5
    )


gs_decision_tree.fit(X_train,y_train)

print(gs_decision_tree.best_params_)

pred = gs_decision_tree.predict(X_test)

print(f'accuracy score {accuracy_score(y_test, pred)}')
print(f'precision {precision_score(y_test, pred,average=None)}')
print(f'recall {recall_score(y_test, pred, average=None)}')

{'criterion': 'gini', 'max_depth': 120, 'min_samples_split': 6, 'min_weight_fraction_leaf': 0.0}
accuracy score 0.35353535353535354
precision [0.         1.         0.         0.8        0.09090909 1.
 1.         1.         0.         1.         0.         0.
 1.         1.         0.66666667 0.66666667 0.78571429]
recall [0.         0.5        0.         1.         0.33333333 0.28571429
 0.66666667 1.         0.         0.2        0.         0.
 0.45454545 0.38095238 0.18181818 0.43478261 0.78571429]


/home/michael/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/michael/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## compounds - flavour

In [15]:
#input layer
inputs3 = keras.Input(compounds.columns.__len__()) 
#hidden layers
h1_3 = keras.layers.Dense(compounds.columns.__len__()*2, activation='relu')
h2_3 = keras.layers.Dense(compounds.columns.__len__(), activation='relu')
output = keras.layers.Dense(flavour.columns.__len__(), activation='sigmoid')

#construct input and hidden layers
x = h1_3(inputs3)
x = h2_3(x)
x = output(x)

model3 = keras.models.Model(inputs = inputs3, outputs = x, name = "Botanical_pred_by_flavour_single_output_layer")

model3.build(compounds.columns.__len__())

model3.compile(
    optimizer=keras.optimizers.Adam(),
    loss = keras.losses.BinaryCrossentropy(),
    metrics = keras.metrics.BinaryAccuracy(),
    run_eagerly=True
)

model3.summary()

Model: "Botanical_pred_by_flavour_single_output_layer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 399)]             0         
                                                                 
 dense_8 (Dense)             (None, 798)               319200    
                                                                 
 dense_9 (Dense)             (None, 399)               318801    
                                                                 
 dense_10 (Dense)            (None, 17)                6800      
                                                                 
Total params: 644801 (2.46 MB)
Trainable params: 644801 (2.46 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
x = []
y = []

#create x and y arrays
for index, row in flavour.iterrows():
    y.append(row.values.tolist())
    
    
for index_1, row in compounds.iterrows():
    x.append(row.values.tolist())

type(x)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [17]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=0.01)
checkpoint = tf.keras.callbacks.ModelCheckpoint('checkpoint.keras', monitor='val_loss', save_best_only=True)
model3.fit(X_train, y_train, validation_data=(X_val, y_val), verbose=True, epochs=150, callbacks=[callback, checkpoint])

Epoch 1/150
7/7 [==============================] - 1s 89ms/step - loss: 0.6293 - binary_accuracy: 0.8468 - val_loss: 0.5239 - val_binary_accuracy: 0.9267
Epoch 2/150
7/7 [==============================] - 0s 71ms/step - loss: 0.4369 - binary_accuracy: 0.9218 - val_loss: 0.3771 - val_binary_accuracy: 0.9267
Epoch 3/150
7/7 [==============================] - 1s 82ms/step - loss: 0.3457 - binary_accuracy: 0.9218 - val_loss: 0.3460 - val_binary_accuracy: 0.9267
Epoch 4/150
7/7 [==============================] - 1s 84ms/step - loss: 0.3105 - binary_accuracy: 0.9218 - val_loss: 0.3126 - val_binary_accuracy: 0.9267
Epoch 5/150
7/7 [==============================] - 1s 74ms/step - loss: 0.2728 - binary_accuracy: 0.9221 - val_loss: 0.2736 - val_binary_accuracy: 0.9267
Epoch 6/150
7/7 [==============================] - 1s 84ms/step - loss: 0.2468 - binary_accuracy: 0.9227 - val_loss: 0.2502 - val_binary_accuracy: 0.9279
Epoch 7/150
7/7 [==============================] - 1s 76ms/step - loss: 0.23

In [18]:
pred_tensor = model3.predict(X_test).round()
# pred_tensor = []
# for p in pred:
#     tensor = np.zeros(len(pred[0]))
#     tensor[p.argmax()] = 1
#     pred_tensor.append(tensor)
print(f'accuracy score {accuracy_score(y_test, pred_tensor)}')
#print(f'balanced accuracy score {balanced_accuracy_score(y_test, pred)}')
print(f'precision {precision_score(y_test, pred_tensor,average=None)}')
print(f'recall {recall_score(y_test, pred_tensor, average=None)}')

3/3 [==============================] - 0s 6ms/step
accuracy score 0.13636363636363635
precision [0.         0.         0.         0.         0.         0.
 0.         0.         0.         1.         0.         0.
 0.         1.         0.         0.75       0.44444444]
recall [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.16666667 0.         0.
 0.         0.28571429 0.         0.3        0.21052632]


/home/michael/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/michael/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# desicion tree

In [70]:
params = {
    #compare both split evaluation functions
    'criterion' : ['gini', 'entropy'], 
    
    #after some research it's was found a tree in this depth range is approate for a database of this size 
    'max_depth' : [20, 40, 60, 80, 100],  
    
    #this describes what % of each class should be at each leaf node, a higher value here can be used to limit overfitting
    'min_weight_fraction_leaf' : [0.0, 0.1, 0.2, 0.3], 
    
    #this decides if a split is allowed based on how many samples are at that node
    'min_samples_split' : [2, 3, 4, 5, 6, 7, 8, 9, 10]
}


gs_decision_tree = GridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid=params,
    scoring='accuracy',
    cv=5
    )


gs_decision_tree.fit(X_train,y_train)

print(gs_decision_tree.best_params_)

pred = gs_decision_tree.predict(X_test)

print(f'accuracy score {accuracy_score(y_test, pred)}')
print(f'precision {precision_score(y_test, pred,average=None)}')
print(f'recall {recall_score(y_test, pred, average=None)}')

{'criterion': 'entropy', 'max_depth': 100, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0}
accuracy score 0.13636363636363635
precision [0.         0.         0.         0.         0.         0.25
 0.5        0.66666667 0.22222222 0.2        0.         0.
 0.         0.33333333 0.         0.28571429 0.22222222]
recall [0.         0.         0.         0.         0.         0.33333333
 0.16666667 0.4        0.33333333 0.2        0.         0.
 0.         0.27272727 0.         0.36363636 0.4       ]


/home/michael/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/michael/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
